In [ ]:
import torch
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertForMaskedLM
import torch.nn as nn


In [ ]:
df = pd.read_csv('/content/drive/My Drive/LIN371/data_with_new_labels.csv')
new_df = df[df['body'].apply(lambda x: len(x.split()) >= 10)]
# want a balanced dataset
# downsample class 0
class_0 = new_df[new_df['label'] == 0]  # Class 0 examples
class_1 = new_df[new_df['label'] == 1]  # Class 1 examples

class_0_downsampled = class_0.sample(n=len(class_1), random_state=2024)
balanced_df = pd.concat([class_0_downsampled, class_1], axis=0)
all_texts = balanced_df['body'].tolist()

from sklearn.model_selection import train_test_split

X = balanced_df['body'].tolist()
y = balanced_df['label'].tolist()
new_labels = balanced_df['new_label'].tolist()

docs_train, docs_test, y_train, y_test, extra_train, extra_test = train_test_split(X, y, new_labels, test_size=0.5, random_state=3, stratify=new_labels)
docs_test, docs_val, y_test, y_val, extra_test, extra_val = train_test_split(docs_test, y_test, extra_test, test_size=0.5, random_state=3, stratify=extra_test)

print ( len(docs_train), len(y_train) )
print ( len(docs_val), len(y_val) )
print ( len(docs_test), len(y_test) )

16439 16439
8220 8220
8219 8219


In [ ]:
import torch

### --- To do: check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
### ------------------------------------

cpu


In [ ]:
from pprint import pprint
from transformers import BertTokenizer, BertModel

### --- To do: BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
if torch.cuda.is_available():
  my_model = BertModel.from_pretrained('bert-base-uncased', device_map={"": 0})
else:
  my_model = BertModel.from_pretrained('bert-base-uncased')
### ------------------------------------

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
exp_words = pd.read_csv('/content/drive/My Drive/LIN371/cleaned_explicit_words.csv')
exp_words_list = exp_words['text'].tolist()

def count_words(words, exp_words_list, counter):
    for word in words:
        if word in exp_words_list:
            counter[word.lower()] = counter.get(word.lower(), 0) + 1  # Increment counter
    return counter

counter = {}
for i,text in enumerate(all_texts):
  counter = count_words(text.split(), exp_words_list, counter)

sorted_counter = dict(sorted(counter.items(), key=lambda item: item[1], reverse=True))

def is_in_bert_vocab(word):
    return word in tokenizer.vocab

in_vocab_words = [word for word, count in sorted_counter.items() if is_in_bert_vocab(word)]
out_of_vocab_words = [word for word, count in sorted_counter.items() if not is_in_bert_vocab(word)]

new_tokens = {word for word in out_of_vocab_words if sorted_counter[word] > 10}

# add the tokens and extend the vocabulary
tokenizer.add_tokens(list(new_tokens))

# add new embeddings
my_model.resize_token_embeddings(len(tokenizer))

NameError: name 'pd' is not defined

In [ ]:
print(sorted_counter)
print(out_of_vocab_words)
print(new_tokens)

{'ass': 931, 'pussy': 749, 'fucking': 611, 'tits': 376, 'cock': 346, 'suck': 336, 'boobs': 130, 'asshole': 115, 'fuck': 109, 'breasts': 76, 'anal': 73, 'finger': 72, 'arse': 60, 'booty': 57, 'asses': 47, 'spanking': 27, 'cunt': 21, 'dicks': 19, 'cumming': 19, 'assholes': 17, 'sperm': 16, 'anus': 15, '69': 13, 'crotch': 12, 'boner': 12, 'cunts': 9, 'sack': 9, 'nip': 9, 'puss': 8, 'fucker': 8, 'douche': 7, 'cums': 7, 'cocks': 7, 'blowjob': 6, 'jizz': 6, 'bbc': 6, 'creampie': 6, 'prick': 5, 'pecker': 5, 'rectum': 5, 'asshat': 4, 'vulva': 4, 'snatch': 3, 'milf': 3, 'douchebag': 3, 'bj': 3, 'groper': 3, 'wanking': 3, 'dickheads': 2, 'clusterfuck': 2, 'dickhead': 2, 'mofo': 2, 'fuckers': 2, 'pricks': 2, 'wang': 2, 'fuk': 2, 'cajones': 2, 'sphincter': 2, 'vag': 2, 'diddle': 2, 'jackasses': 2, 'fcking': 1, 'peen': 1, 'cameltoe': 1, 'fukk': 1, 'fck': 1, 'bellend': 1, 'arsehole': 1, 'cooter': 1, 'dong': 1, 'bates': 1, 'pussys': 1, 'azz': 1, 'fuckstick': 1, 'octopussy': 1, 'doosh': 1, 'jizzing': 

In [ ]:
import random
import numpy as np

def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
### --- To do: construct a Dataset class
class Dataset(torch.utils.data.Dataset):
  def __init__(self, docs, ys):
    self.labels = ys
    self.texts = [
        tokenizer(
            text, padding='max_length', max_length=512, truncation=True, return_tensors='pt'
        ) for text in docs
    ]

  def classes(self):
    return self.labels

  def __len__(self):
    return len(self.labels)

  def get_batch_labels(self, idx):
    return np.array(self.labels[idx])

  def get_batch_texts(self, idx):
    return self.texts[idx]

  def __getitem__(self,idx):
    batch_texts = self.get_batch_texts(idx)
    batch_ys = self.get_batch_labels(idx)
    return batch_texts, batch_ys
### ------------------------------------

In [ ]:
### --- To do: construct BertClassifier
import torch
import torch
from transformers import BertTokenizer, BertModel, BertConfig

class BertClassifier(torch.nn.Module):
    def __init__(self, tokenizer=None, saved_model_path=None, dropout=0.2, freeze_layer_count=0):
        super(BertClassifier, self).__init__()

        # If a saved model path is provided, load the pre-trained BERT model from the checkpoint
        if saved_model_path:
            if torch.cuda.is_available():
                self.bert = BertModel.from_pretrained(saved_model_path, device_map={"": 0})
            else:
                self.bert = BertModel.from_pretrained(saved_model_path)
            self.tokenizer = BertTokenizer.from_pretrained(saved_model_path)
        else:
            # If no saved model path, just initialize the model from scratch
            self.tokenizer = tokenizer
            if torch.cuda.is_available():
                self.bert = BertModel.from_pretrained('bert-base-uncased', device_map={"": 0})
            else:
                self.bert = BertModel.from_pretrained('bert-base-uncased')
            self.bert.resize_token_embeddings(len(self.tokenizer))


        # Freeze layers if specified
        if freeze_layer_count:
            for param in self.bert.embeddings.parameters():
                param.requires_grad = False
            if freeze_layer_count != -1:
                for layer in self.bert.encoder.layer[:freeze_layer_count]:
                    for param in layer.parameters():
                        param.requires_grad = False

        # Dropout and final classifier layer
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(768, 2)  # Assuming 2 output labels

    def forward(self, input_id, mask):
        # Pass through BERT
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)

        # Apply dropout
        dropout_output = self.dropout(pooled_output)

        # Final linear layer (classifier output)
        linear_output = self.linear(dropout_output)

        return linear_output

    def save_model(self):
        model_repo_name = "dxhf100/my-pytorch-model"

        save_path = '/content/drive/My Drive/LIN371/best_model/'

        # Push model and tokenizer to the Hub
        self.bert.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)
        print(f"Model and tokenizer saved to {save_path}")

        self.bert.push_to_hub(model_repo_name)
        self.tokenizer.push_to_hub(model_repo_name)
        print(f"Model and tokenizer pushed to the Hub at {model_repo_name}")


### ------------------------------------

In [ ]:
from sklearn.metrics import f1_score, classification_report

### --- To do: construct an evaluation function
def evaluate(model, eval_dataloader):
  total_acc = 0
  total_loss = 0

  criterion = torch.nn.CrossEntropyLoss()

  eval_labels, eval_preds = [], []
  with torch.no_grad():
    for eval_input, eval_label in eval_dataloader:
      input_ids = eval_input['input_ids'].squeeze().to(device)
      attention_mask = eval_input['attention_mask'].to(device)
      eval_label = eval_label.type(torch.LongTensor).to(device)

      output = model(input_ids, attention_mask)
      batch_loss = criterion(output, eval_label)
      total_loss += batch_loss.item()

      acc = (output.argmax(dim=1) == eval_label).sum().item()
      total_acc += acc

      eval_labels.extend(eval_label.tolist())
      eval_preds.extend(output.argmax(dim=1).tolist())

    f1_macro = f1_score(y_true=eval_labels, y_pred=eval_preds, average='macro')
    report = classification_report(y_true=eval_labels, y_pred=eval_preds)
  return total_acc, total_loss, f1_macro, report
### ------------------------------------

In [ ]:
'''
starting training...
====================
 Epoch  |  val acc  |  val f1
--------------------------------------------------

   1    |   0.921   |   0.921
New best f1 0.9209155103526525 found at epoch 1. Saving model.
Model and tokenizer saved to /content/drive/My Drive/LIN371/best_model/

model.safetensors: 100%
 438M/438M [00:26<00:00, 23.6MB/s]

Model and tokenizer pushed to the Hub at dxhf100/my-pytorch-model

   2    |   0.921   |   0.921

   3    |   0.921   |   0.921
New best f1 0.9210181454784354 found at epoch 3. Saving model.
Model and tokenizer saved to /content/drive/My Drive/LIN371/best_model/
'''

In [ ]:
### --- To do: Using the LOADED BERT
set_seed(3)

loaded_model = BertClassifier(saved_model_path="/content/drive/My Drive/LIN371/best_model/", freeze_layer_count=-1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

test = Dataset(docs_test, y_test)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=16)
report = evaluate(loaded_model, test_dataloader)
for p in report:
  print(p)
### ------------------------------------

7546
129.15164394443855
0.918094133034637
              precision    recall  f1-score   support

           0       0.90      0.93      0.92      4110
           1       0.93      0.90      0.92      4109

    accuracy                           0.92      8219
   macro avg       0.92      0.92      0.92      8219
weighted avg       0.92      0.92      0.92      8219



In [ ]:
explicit_source_no_explicit_words = [text for i, text in enumerate(docs_test) if extra_test[i] == 'explicit_source_no_explicit_words']
explicit_source_has_explicit_words = [text for i, text in enumerate(docs_test) if extra_test[i] == 'explicit_source_has_explicit_words']
control_source_no_explicit_words = [text for i, text in enumerate(docs_test) if extra_test[i] == 'control_source_no_explicit_words']
control_source_has_explicit_words = [text for i, text in enumerate(docs_test) if extra_test[i] == 'control_source_has_explicit_words']

In [ ]:
### --- To do: Using the LOADED BERT
set_seed(3)

test1 = Dataset(explicit_source_no_explicit_words, [1]*len(explicit_source_no_explicit_words))
test_dataloader1 = torch.utils.data.DataLoader(test1, batch_size=16)
report = evaluate(loaded_model, test_dataloader1)
for p in report:
  print(p)

### ------------------------------------

2387
56.18132441281341
0.4694198623402163
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.88      0.94      2698

    accuracy                           0.88      2698
   macro avg       0.50      0.44      0.47      2698
weighted avg       1.00      0.88      0.94      2698



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(2387 / len(explicit_source_no_explicit_words))

0.8847294292068199


In [ ]:
### --- To do: Using the LOADED BERT
set_seed(3)

test2 = Dataset(explicit_source_has_explicit_words, [1]*len(explicit_source_has_explicit_words))
test_dataloader2 = torch.utils.data.DataLoader(test2, batch_size=16)
report = evaluate(loaded_model, test_dataloader2)
for p in report:
  print(p)

### ------------------------------------

1327
15.35046717885416
0.48466033601168734
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.94      0.97      1411

    accuracy                           0.94      1411
   macro avg       0.50      0.47      0.48      1411
weighted avg       1.00      0.94      0.97      1411



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(1327 / len(explicit_source_has_explicit_words))

0.9404677533664068


In [ ]:
### --- To do: Using the LOADED BERT
set_seed(3)

test3 = Dataset(control_source_no_explicit_words, [0]*len(control_source_no_explicit_words))
test_dataloader3 = torch.utils.data.DataLoader(test3, batch_size=16)
report = evaluate(loaded_model, test_dataloader3)
for p in report:
  print(p)

### ------------------------------------

3255
44.59946377994493
0.483942908117752
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      3471
           1       0.00      0.00      0.00         0

    accuracy                           0.94      3471
   macro avg       0.50      0.47      0.48      3471
weighted avg       1.00      0.94      0.97      3471



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(3255 / len(control_source_no_explicit_words))

0.9377700950734659


In [ ]:
### --- To do: Using the LOADED BERT
set_seed(3)

test4 = Dataset(control_source_has_explicit_words, [0]*len(control_source_has_explicit_words))
test_dataloader4 = torch.utils.data.DataLoader(test4, batch_size=16)
report = evaluate(loaded_model, test_dataloader4)
for p in report:
  print(p)

### ------------------------------------

584
13.015775698237121
0.47751430907604253
              precision    recall  f1-score   support

           0       1.00      0.91      0.96       639
           1       0.00      0.00      0.00         0

    accuracy                           0.91       639
   macro avg       0.50      0.46      0.48       639
weighted avg       1.00      0.91      0.96       639



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(584 / len(control_source_has_explicit_words))

0.9139280125195618


In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, BertForMaskedLM
import torch.nn as nn

tokenizer = BertTokenizer.from_pretrained("/content/drive/My Drive/LIN371/best_model/")
model = BertModel.from_pretrained("/content/drive/My Drive/LIN371/best_model/")

In [ ]:
import random

def get_valid_words(tokenizer):
    vocab_keys = list(tokenizer.get_vocab().keys())
    valid_words = [
        word for word in vocab_keys
        if not word.startswith("##") and word.isalpha()
    ]
    return valid_words

def replace_words(sentence, replace_word, exp_words_list, seed=10):
    sentences = []
    new_words = []
    valid_words = get_valid_words(tokenizer)
    # Replace with words from exp_words_list
    for word in exp_words_list[:50]:
        new_sentence = sentence.replace(replace_word, word)
        sentences.append(new_sentence)
        new_words.append(word)

    # Replace with random words from tokenizer's vocabulary
    random.seed(seed)
    for _ in range(50):
        random_word = random.choice(valid_words)  # Randomly select a word from vocab
        new_sentence = sentence.replace(replace_word, random_word)
        sentences.append(new_sentence)
        new_words.append(random_word)

    return sentences, new_words

In [ ]:
sentence = "She's got some perky peaks, you know."
exp_words_list = [word for word, count in sorted_counter.items()]
sentences, new_words = replace_words(sentence, 'peaks', exp_words_list, seed=10)
for sentence, new_word in zip(sentences[:10], new_words[:10]):
  print(sentence, "\t", new_word)

NameError: name 'sorted_counter' is not defined

In [ ]:

def get_embeddings_for_cos(sentences, new_words, sentence, word):

  with torch.no_grad():
    # get the original sentence
    tokenized_sent = [
          tokenizer(
              sentence, padding='max_length', max_length=512, truncation=True, return_tensors='pt'
          )]
    input_ids = tokenized_sent[0]['input_ids'].to(device)
    attention_mask = tokenized_sent[0]['attention_mask'].to(device)
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_state = output.last_hidden_state
    tokens = tokenizer.tokenize(sentence)
    self_tokens = tokenizer.tokenize(word)
    word_indices = []
    for j in range(len(self_tokens)):
      word_indices.append(tokens.index(self_tokens[j]))
    first_embedding = last_hidden_state[0, word_indices, :].mean(dim=0)

    word_embeddings = []
    for i, sent in enumerate(sentences):

      tokenized_sent = [
        tokenizer(
            sent, padding='max_length', max_length=512, truncation=True, return_tensors='pt'
        )]

      input_ids = tokenized_sent[0]['input_ids'].to(device)
      attention_mask = tokenized_sent[0]['attention_mask'].to(device)
      new_word = new_words[i]

      output = model(input_ids=input_ids, attention_mask=attention_mask)
      last_hidden_state = output.last_hidden_state

      # Get wordpiece tokens
      tokens = tokenizer.tokenize(sent)

      # word by itself
      self_tokens = tokenizer.tokenize(new_word)

      # Find the indices for subwords of a word (e.g., 'running')
      word_indices = []
      for j in range(len(self_tokens)):
        word_indices.append(tokens.index(self_tokens[j]))

      # Average embeddings for subwords
      word_embedding = last_hidden_state[0, word_indices, :].mean(dim=0)
      word_embeddings.append(word_embedding)

    return first_embedding, word_embeddings


In [ ]:
first_embedding, word_embeddings = get_embeddings_for_cos(sentences[90:], new_words[90:], sentence, 'peaks')

In [ ]:
import torch.nn.functional as F

for i, embedding in enumerate(word_embeddings):
  similarity = F.cosine_similarity(first_embedding, embedding, dim=0)
  print(similarity, new_words[i])

tensor(0.9190) ass
tensor(0.9293) pussy
tensor(0.8256) fucking
tensor(0.9002) tits
tensor(0.8773) cock
tensor(0.9116) suck
tensor(0.8243) boobs
tensor(0.8228) asshole
tensor(0.7806) fuck
tensor(0.8741) breasts


In [ ]:
import torch.nn.functional as F

for i, embedding in enumerate(word_embeddings):
  similarity = F.cosine_similarity(first_embedding, embedding, dim=0)
  print(similarity, new_words[90+i])

tensor(0.5366) lucius
tensor(0.5397) songs
tensor(0.4234) overrun
tensor(0.7778) х
tensor(0.3774) vegetation
tensor(0.8113) components
tensor(0.7560) equation
tensor(0.6535) barrett
tensor(0.8129) sweetly
tensor(0.5360) vowels


In [ ]:
sentence = "Once we climb these mountain peaks, we'll be able to see the sunset."
exp_words_list = [word for word, count in sorted_counter.items()]
sentences, new_words = replace_words(sentence, 'peaks', exp_words_list, seed=90)
sentences[:10], new_words[:10]

(["Once we climb these mountain ass, we'll be able to see the sunset.",
  "Once we climb these mountain pussy, we'll be able to see the sunset.",
  "Once we climb these mountain fucking, we'll be able to see the sunset.",
  "Once we climb these mountain tits, we'll be able to see the sunset.",
  "Once we climb these mountain cock, we'll be able to see the sunset.",
  "Once we climb these mountain suck, we'll be able to see the sunset.",
  "Once we climb these mountain boobs, we'll be able to see the sunset.",
  "Once we climb these mountain asshole, we'll be able to see the sunset.",
  "Once we climb these mountain fuck, we'll be able to see the sunset.",
  "Once we climb these mountain breasts, we'll be able to see the sunset."],
 ['ass',
  'pussy',
  'fucking',
  'tits',
  'cock',
  'suck',
  'boobs',
  'asshole',
  'fuck',
  'breasts'])

In [ ]:
first_embedding, word_embeddings = get_embeddings_for_cos(sentences[90:], new_words[90:], sentence, 'peaks')

In [ ]:
import torch.nn.functional as F

for i, embedding in enumerate(word_embeddings):
  similarity = F.cosine_similarity(first_embedding, embedding, dim=0)
  print(similarity, new_words[i])

tensor(0.4286) ass
tensor(0.3056) pussy
tensor(0.8152) fucking
tensor(0.3181) tits
tensor(0.5972) cock
tensor(0.7017) suck
tensor(0.8676) boobs
tensor(0.7341) asshole
tensor(0.7526) fuck
tensor(0.5257) breasts


In [ ]:
import torch.nn.functional as F

for i, embedding in enumerate(word_embeddings):
  similarity = F.cosine_similarity(first_embedding, embedding, dim=0)
  print(similarity, new_words[90+i])

tensor(0.7244) peacekeeping
tensor(0.6937) radioactive
tensor(0.7557) powerful
tensor(0.7727) य
tensor(0.6777) м
tensor(0.8019) quiet
tensor(0.7737) toes
tensor(0.7434) templar
tensor(0.6483) clinton
tensor(0.7964) work
